In [1]:
import unittest
from blinker import signal, Signal
import silq
silq.initialize('Serwan')

Initializing imports
Initialization complete


C:\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
from silq.tools.config import *

In [4]:
class TestPulseSignals(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse')
        self.is_set = False 
        self.dict = {}

    def test_set_val(self):
        self.assertFalse(self.is_set)
        self.set_val(self)
        self.assertTrue(self.is_set)

    def test_signal_emit(self):
        p = Pulse()
        self.signal.connect(self.set_dict)
        self.assertFalse('t_start' in self.dict)
        p.t_start = 1
        self.assertEqual(1, self.dict['t_start'])

    def test_subsequent_pulses(self):
        p1 = Pulse(t_start=0, t_stop=10)
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 10)
        self.assertEqual(p1.t_stop, 10)
        
        p2 = Pulse(t_start=PulseMatch(p1, 't_stop', delay=1), duration=4)
        self.assertEqual(p2.t_start, 11)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 15)
        
        p1.t_stop = 14
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 14)
        self.assertEqual(p1.t_stop, 14)
        
        self.assertEqual(p2.t_start, 15)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 19)
        
        p2.t_start = 0
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)
        
        p1.t_stop = 20
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)

    def set_val(self, sender, **kwargs):
        self.is_set = True
        
    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [5]:
class TestPulseConfig(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse_config:read')
        
        pulses_dict = {'read': {}}
        self.pulses_config = DictConfig(name='pulses',
                                        filepath=None,
                                        config=pulses_dict,
                                        item_class=PulseConfig)
        qc.config.user.pulses = self.pulses_config
        self.pulse_config = self.pulses_config['read']
        self.dict = {}
        Pulse.pulses_config = self.pulses_config

    def test_set_item(self):
        with self.assertRaises(KeyError):
            self.pulse_config['duration']
        self.pulse_config['duration'] = 1
        self.assertEqual(self.pulse_config['duration'], 1)
        self.assertEqual(self.pulse_config.duration, 1)

    def test_signal(self):
        self.signal.connect(self.set_dict)
        self.pulse_config.duration = 1
        self.assertIn('duration', self.pulse_config)
        self.assertEqual(self.pulse_config.get('duration', None), 1)

    def test_pulse_from_config(self):
        self.pulse_config.t_start = 10
        p = Pulse(name='read')
        self.assertEqual(p.t_start, 10)
        
        p.t_start = 20
        self.assertEqual(p.t_start, 20)
        
        self.pulse_config.t_start = 0
        self.assertEqual(p.t_start, 0)
        
        self.pulse_config.t_start = 'config:pulses.read.t_stop'
        with self.assertRaises(AttributeError):
            self.pulse_config.t_start
            
        self.pulse_config.t_stop = 50
        self.assertEqual(self.pulse_config.t_start, 50)

    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [6]:
# Run tests    
suite_pulse_signals = unittest.TestLoader().loadTestsFromModule(TestPulseSignals())
suite_pulse_config = unittest.TestLoader().loadTestsFromModule(TestPulseConfig())
suite = unittest.TestSuite([suite_pulse_signals, suite_pulse_config])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_set_val (__main__.TestPulseSignals) ... ok
test_signal_emit (__main__.TestPulseSignals) ... ok
test_subsequent_pulses (__main__.TestPulseSignals) ... ok
test_pulse_from_config (__main__.TestPulseConfig) ... ERROR

ERROR: test_pulse_from_config (__main__.TestPulseConfig)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-5-ea20e9c744c5>", line 43, in test_pulse_from_config
    self.pulse_config.t_stop = 50
  File "c:\users\serwan\documents\github\silq\silq\tools\config.py", line 57, in __setitem__
    self.signal.send(self, **{key: val})
  File "C:\Anaconda3\lib\site-packages\blinker\base.py", line 267, in send
    for receiver in self.receivers_for(sender)]
  File "C:\Anaconda3\lib\site-packages\blinker\base.py", line 267, in <listcomp>
    for receiver in self.receivers_for(sender)]
  File "c:\users\serwan\documents\github\silq\silq\pulses\pulse_types.py", line 75, in _handle_config_signal
    setattr(s

<unittest.runner.TextTestResult run=4 errors=1 failures=0>

In [7]:


pulses_dict = {'read': {}}
pulses_config = DictConfig(name='pulses',
                                filepath=None,
                                config=pulses_dict,
                                item_class=PulseConfig)
qc.config.user.pulses = pulses_config
pulse_config = pulses_config['read']
dict = {}
Pulse.pulses_config = pulses_config
p = Pulse(name='read')

In [10]:
pulse_config.t_start = 'config:pulses.read.t_stop'

In [13]:
pulse_config.__dict__

{'name': 'read',
 'signal': <blinker.base.NamedSignal object at 0x000001B286B8C160; 'pulse_config:read'>}

In [9]:
pulse_config.t_stop = 50

TypeError: unsupported operand type(s) for -: 'int' and 'str'